In [22]:
import torch
import os 
import sklearn 
import pandas as pd 
import numpy as np
from carla.data.catalog import CsvCatalog

In [25]:
#TODO How to get access to test set from CARLA ? 
name='credit'
dataset = pd.read_csv(f'../data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
continuous_wachter = dataset.drop(columns=['label']).columns
dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
dataset._df_test

,label,x1,x2,x3,x4,x6,x5,x7
5836,0.0,1.0,14.101458,-0.063512,0.886963,1.350893,6.058314,-5.195142
837,0.0,0.0,18.131641,-0.034312,0.050784,-1.482156,1.126881,-3.835941
9249,1.0,0.0,8.379956,0.020689,2.032427,1.212866,3.158215,2.422019
344,0.0,0.0,-3.645628,-0.132162,0.178333,-0.001406,2.833725,-2.589821
1869,1.0,1.0,-4.374931,-0.041227,0.661066,0.567683,1.625891,8.408765
...,...,...,...,...,...,...,...,...
3374,0.0,1.0,1.759700,0.023923,3.382517,-3.497670,8.814781,0.416418
9449,1.0,0.0,-19.059351,-0.201270,-4.929326,-4.110567,-9.940821,-8.278362
372,1.0,0.0,3.920522,-0.163596,-0.555549,1.553838,-1.913766,5.128632
2069,0.0,0.0,4.301075,-0.050205,3.376857,-1.778547,6.670550,-12.324182


In [31]:
model_name=['0']
accuracy=['0']
precision=['0']
recall=['0']
df=pd.DataFrame([])
for name in os.listdir('./Model/'):
    full_name=name
    model=torch.load(f'./Model/{name}')
    name=name.split('_')[1]
    name=name.replace('.pth','')
    dataset = pd.read_csv(f'../data/{name}/{name}.csv')
    continuous_wachter = dataset.drop(columns=['label']).columns
    dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
    dataset=dataset._df_test
    data_x, data_y=dataset.drop(columns=['label']), dataset['label']
    data_pred= model.predict(data_x)
    model_name.append(full_name)
    print(data_pred.shape)
    print(data_y.shape)
    if len(data_pred.shape)>1:
        data_pred= np.argmax(data_pred, axis=1)

    accuracy.append(sklearn.metrics.accuracy_score(data_y,data_pred))
    precision.append(sklearn.metrics.precision_score(data_y,data_pred))
    recall.append(sklearn.metrics.recall_score(data_y,data_pred))
df['model_name']=model_name
df['accuracy']=accuracy
df['precision']=precision
df['recall']=recall
df

(2500,)
(2500,)
(2500,)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


,model_name,accuracy,precision,recall
0,0,0,0,0
1,Forest_credit.pth,0.802,0.827842,0.778466
2,Forest_sanity-3-lin.pth,0.81,0.847144,0.786996
3,Linear_credit.pth,0.7428,0.77284,0.718989
4,Linear_sanity-3-lin.pth,0.9968,0.996958,0.996958
5,Linear_sanity-3-non-add.pth,0.9992,1,0.99689
6,Linear_sanity-3-non-lin.pth,0.9984,0.999141,0.997427
7,MLP_credit.pth,0.688,0.70646,0.672374
8,MLP_sanity-3-lin.pth,0.9968,0.999217,0.994544
9,MLP_sanity-3-non-add.pth,0.9908,0.989967,0.972085
